In [1]:
import numpy as np
import cv2 
import sys 
import matplotlib.pyplot as plt


## 특징점 검출과 매칭\


1. 코너 검출\

코너의 특징\
- 평탄한 영역 (flat) & 에지(edge) 영역은 고유한 위치를 찾기 어려움\
- 코너는 변별력이 높은 편이며, 영상의 이동 회전 변환에 강인함\

<코너 검출 방법>\

해리스 코너 검출 방법 (harris):\
영상 내부의 작은 영역이 모든 방향에 대해 변화가 큰 경우 코너로 규정\
코너 응답함수 R은 반환=> R(x,y)가 충분히 크면 코너로 구분\

cv2.cornerHarris() 함수 사용\




추적하기 좋은 특징(Good Features to Track):\
해리스 코너 검출 방법을 기반으로 향상된 방법\
비최대 억제 수행\
코너 품질 함수를 정의 => 가장 값이 큰 순서대로 정렬하여 반환\

cv2.goodFeaturesToTrack()함수 사용

FAST (Features from Accelerated Segment Test):\
주변 16개 픽셀값 크기를 분석\
기준 픽셀(p)보다 충분히 밝거나 (>p+t) 또는 충분히 어두운 (<p-t)\
픽셀이 n개 연속으로 나타나면 코너로 인식(n은 보통9)\
해리스 GIFT방법 보다 매우 빠르게 동작\


해리스 코너 응답 함수 계산\
cv2.cornerHarris(src, blockSize, ksize, k, borderType=None)=> dst\

src = 입력 단일 채널 8비트 또는 실수형 영상\
blockSize =코너 응답 함수 계산에서 고려할 이웃 픽셀 크기 , 보통 2-5\
ksize:(미분을 위한) 소벨 연산자를 위한 커널 크기. 보통 3\
k = 해리스 코너 검출 상수(보통 0.04~0.06)\
borderType= 가장자리 픽셀 확장 방식. 기본값=> cv2.BORDER_DEFAULT\
dst = 해리스 코너 응답 계수. src와 같은 크기의 행렬\

추적하기 좋은 특징 코너 검출\
cv2.goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance, mask=None, blockSize=None, useHarrisDetector = None, k=None)=> corners\

image: 8비트 또는 32비트 실수 , 단일 채널 영상\
maxCorners:최대 코너 개수. \
qualityLevel:코너점 결정을 위한 값\
minDistance = 코너점 사이 최소 거리\
corners: 검출된 코너점 좌표 .\
mask: 마스크 영상\
blockSize:코너 검출을 위한 블록 크기. 기본값3 \
useHarrisDetector:해리스 코너 방법 사용 여부\
k : 해리스 코너 검출시 사용할 값\

FAST 코너 검출\
cv2.FastFeatureDetector_create(threshold=None, nonmaxSuppression=None,\
type=None)=>> retval\

cv2.FastFeatureDetector.detect(image)=> keypoints\
threshold: 중심 픽셀값과 주변 픽셀값과의 차이 임계값. 기본값 10 \
nonmaxSuppression:비최대 억제 수행 여부. 기본값은 True\
type:코너 검출 방법. 기본값, 
retval:FastFeatureDetector 객체\
image:그레이 스케일 영상
keypoints(출력):검출된 코너점 정보,cv2.KeyPoint 객체를 담은 리스트.\
cv2.KeyPoint의 pt 멤버를 이용하여 코너 좌표 추출. \
pt[0], pt1[], (x,y)좌표\

코너 검출 반복성 비교\
FAST방법의 반복 검출률이 대체로 높다\
FAST방법은 노이즈에 민감함

In [1]:
import numpy as np 
import cv2 
import sys 
import matplotlib.pyplot as plt

src = cv2.imread('building.jpg',cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed')
    sys.exit()
    
tm = cv2.TickMeter() # 연산 시간 측정 개체

# GIFT
tm.start()


# 특정 코너 좌표 얻기
# 이미지, 코너검출 최대수 , 코너결정상수, 코너사이 최소거리
corners = cv2.goodFeaturesToTrack(src, 400, 0.01, 10)

# 시간 측정 종료
tm.stop()
print('GFTT: {}ms.'.format(tm.getTimeMilli()))

dst1 = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

#코너를 빨간점으로 그려라 
if corners is not None:
    for i in range(corners.shape[0]):
        # 각각의 코너 좌표 정의
        pt = (int(corners[i,0,0]), int(corners[i,0,1]))
        cv2.circle(dst1,pt,5,(0,0,255),2)
        

# FAST
tm.reset()
tm.start()

cv2.imshow('src',src)
cv2.imshow('dst1',dst1)
cv2.waitKey()

cv2.destroyAllWindows()



GFTT: 49.8436ms.


In [3]:

# FAST
tm.reset()
tm.start()

#Fast 코너 검출 이용
fast = cv2.FastFeatureDetector_create(60) # 괄호안에 60은 임계값

# fastFeature함수 이용 그리고 그 안의 기능 detect로 src 이미지읽어서
# fast코너 좌표 얻기
keypoints = fast.detect(src)

tm.stop()
print('FAST: {}ms.'.format(tm.getTimeMilli()))

# 그레이 스케일 이미지를 받기 위해서 원본이미지 그레이스케일로 바꿔줌\
dst2 =cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

# 모든 FAST코너검출로 인한 코너들의 좌표들에게
for kp in keypoints:
    #각각의 코너 좌표 정의
    # 여기서 이미 keypoints는 객체 여러개로 출력됨
    pt = (int(kp.pt[0]), int(kp.pt[1]))
    # 이 코너 좌표들에게 빨간 동그라미 씌여줌
    cv2.circle(dst2, pt, 5, (0,0,255), 2)
    
cv2.imshow('dst2',dst2)
cv2.waitKey()
cv2.destroyAllWindows()

FAST: 7.1061ms.


### 비교 결과\
추적하기 좋은 코너 검출 방법이 더 빠르다\
허나 FAST 코너 검출 방법이 느리지만 더 많은 코너좌표들을 가져왓고 \
활용 의도에 맞게 사용하는것 같다

===================================================================

2. 이미지 피라미드 \
이미지를 확대, 축소 샘플링(4배, 1/4배 등)\
동작:\
upsize(zoom in ) / downsize(zoom out)\


[Gaussian pyramid]\
1) Gaussian Pyramid:이미지 다운 샘플링=> 즉 화면 축소 \
모든 레이어는 아래에서 위로 번호가 매겨짐 (i+1)\
짝수번째 행과 열을 제거함으로 축소\


2) Laplacian Pyramid: 위 가우시안 피라미드 아래층의 이미지로 업 샘플링된\
이미지 재구성(해상도가 낮다)}

Gaussian Pyramid 예\
img = cv2.imread(jpg)
lower_reso = cv2.pyrDown(img) # 원본 이미지의 1/4이즈
higer_reso = cv2.pyrUp(img) # 원본 이미지의 4배 사이즈\



Laplacian Pyramid 예\
가우시안 피라미드로 이미지를 축소했다가 다시 확대하면 이 과정에서 행과 열이 소실된다. 이렇게 생성된 이미지를 원본 크기로 변경한뒤 \
원본 이미지랑 피라미드 이미지의 차이를 계산하면 이미지 윤곽이 도출됨

In [4]:
img = cv2.imread('building.jpg')

s = cv2.pyrDown(img) # 원본 이미지의 1/4사이즈
l = cv2.pyrUp(s) #원본 이미지의 4배 사이즈

#이미지 사이즈
h,w,c = img.shape

# 원본 이미지의 4배사이즈가 되었던걸로 다시 원본 이미지 크기로 크기 변경
l2 = cv2.resize(l, (w,h))

# 원본이미지랑 피라미드 이미지의 차이를 계산하면 이미지 윤곽 도출
# 원본이미지 에서 피라미드이미지 했다가 다시 크기변경한거를 뺴줌
res = cv2.subtract(img, l2)

# 윤곽 확인 가능
cv2.imshow('img',res)
cv2.waitKey(0)
cv2.destroyAllWindows()


3. 특징점 검출\

Harris, GFTT, FAST 코너 검출 방법의 문제점은?\
장점=> 이동, 회전 변환에 강인\
단점: 크기 변환에 취약\
=> 다양한 크기 관점에서의 특징 검출 필요\


크기 불변 특징점 검출 방법\
SIFT, KAZE, AKAZE ORB 등 다양한 특징점 검출 방법에서 \
스케일 스페이스(scale-space)\
이미지 피라미드 (image-pyramid)\
를 구성하여 크기 불변 특징점을 검출.\

크기 불변 특징점이란?\

영상 데이터는 수치적으로 보았을떄\
2차원 영상을 받아들이는 디바이스가 조금이라도 흔들리면 다른 데이터가 된다\

영상 제대로 분석하기 위해서는 픽셀 좌표를 비교하려는 이미지와 동일한 위치에 오도록 수정하여 분석가능하다\

이점이 굉장히 불편하다, \

예)\
=> 눈앞 1미터 위치에 있을때만 친구 얼굴을 친구라고 인식 가능, \
그 위치에서 1센치라도 움직이면 친구라고 인식 못함\

그러므로 이미지에서 특징 추출해서, 추출한 특징 정보에서 => 또다른 정보 추츌\



하지만 특징 중에서도 이미지 크기에 따라 영향 받는게 존재\
그래서 스케일을 무시 못한다\
그래서 크기가 다른 영상에서도 특징 검출을 할수 있게 만든 알고리즘이\
=> Scale Invariant Feature Transform\
SIFT : 크기 불변 특징 변환, \
=> 영상 크기변화에 무관하게 특징점 추출하기 위해서, \
scale space를 구성한다, \

스케일 스페이스는 영상에 다양한 표준 편차를 이용한 가우시안 블러링을 이용하여 만든 영상 집합.


여기서 , 이 스케일 스페이스 에서 SIFT 알고리즘이 크기에 무관한 특징 추출하려면\
=> DOG(Differenct of Gaussian) ,즉 가우시안 차를 만들어야 된다\
가우시안 차는, 인접한 블러링 영상과의 픽셀값 차이를 구한것\


이 가우시안 차를 이용해서 DOG 영상 집합을 구성하고, \
=> 에지 성분이 강하거나 명암비가 낮은 지점은 특징점에서 제외된다\
특징적인 값이 특징점으로 나타나고, 코너가 아닌 특징을 제거한다\
이게 특징점을 추출하는 방식

KAZE 알고리즘 \
그냥 그런게 있다\


ORB(Oriented FAST and ROtated BRIEF) 알고리즘\
- 코너 검출시 FAST코너 검출방식(FAST 코너 검출방식: 특정좌표 주변 16개 픽셀 분석으로 코너 검출)\
- FAST 코너 검출은 크기 변화에 취약\
- ORB는 입력영상의 크기를 점진적으로 축소시킨 피라미드 영상 구축\
- 그 피라미드 영상에 대해 FAST 기법으로 코너 특징점 구해낸다
- 그리고 구해낸 후에, 각 특징점별 주된 방향 성분을 계산하여 BRIEFT알고리즘으로 이진 기술자를 계산\

BRIEF 알고리즘 => 특징점 주변의 픽셀 쌍을 미리 정하고 해당 픽셀값 크기를 0또는 1로 특징을 기술 \

r(x,y) = if 1 I(x) < I(y) 일때 else 0 \
특정좌표끼리 더 밝은지 아니면 어두운지 1과 0으로 표현

특징점 검출 알고리즘 객체 생성\
cv2.KAZE_create() -> retval\
cv2.AKAZE_create() -> retval\
cv2.ORB_create() -> retval\
cv2.xfeatures2d.SIFT_create() -> retval\
...\
retval:각 특징점 검출 알고리즘 객체

특징점 검출 함수 \
cv2.Feature2D.detect(image, mask=None) => keypoints(특징점 좌표)\

image = 입력 영상\
mask= 마스크 영상\
keypoints = 검출된 특징점 정보, cv2.KeyPoint 객체의 리스트

검출된 특징점 그리기 함수\
cv2.drawKeypoints(image, keypoints, color=None, flags=None)\
=> outImage\

image:입력영상\
keypoints:검출된 특징점 정보. cv2.KeyPoint객체의 리스트\
color: 특징점 표현 색상, 임의의 색상이 기본값\
flags:특징점 표현 방법\

flag종류\
cv2.DRAW_MATCHES_FLAGS_DEFAULT:특징점 위치만을 표현하는 작은 크기 원\
cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS:특징점의 크기와 방향을 반영한 원\

outImage:출력 영상

In [7]:
# 영상 불러오기

#같은 사진이지만 관점이 조금 다른 두 사진
src1 = cv2.imread('graf1.png',cv2.IMREAD_GRAYSCALE)
src2 = cv2.imread('graf3.png',cv2.IMREAD_GRAYSCALE)


if src1 is None or src2 is None:
    print('Image load failed!')
    sys.exit()
    
# 특징점 알고리즘 객체 생성(KAZE,AKAZE,ORB)
# feature = cv2.KAZE_create()
#feature = cv2.AKAZE_create()
feature = cv2.ORB_create()


# 특징점 검출
# 특징점 좌표를 얻는다
kp1 = feature.detect(src1)
kp2 = feature.detect(src2)

#특징점들은 여러개로 나타날것이므로 특징점들의 개수 확인
print('# of kp1:',len(kp1))
print('# of kp2:', len(kp2))

# 검출된 특징점 출력 영상 생성
# cv2.drawKeypoints(image, keypoints, color=None, flags=None)

dst1 = cv2.drawKeypoints(src1, kp1, None,
                        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
dst2 = cv2.drawKeypoints(src2, kp2, None,
                        flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


# 이미지로 확인
cv2.imshow('dst1',dst1)
cv2.imshow('dst2',dst2)
cv2.waitKey()
cv2.destroyAllWindows()


# of kp1: 500
# of kp2: 500


4 .특징점 기술\

기술자 (descriptor, feature Vector)\
=> 특징점 근방의 부분 영상을 표현하는 실수 또는 이진벡터\
=>opencv에서는 2차원 행렬(numpy.ndarray)로 표현\
행 개수: 특징점 개수\
열 개수: 특징점 기술자 알고리즘에 의해 정의됨\
실수 기술자: numpy.float32\
이진 기술자: numpy.uint8\

실수 기술자\
=> 특징점 부근 영상의, 방향 히스토그램 사용\
=> 특징점 근방 부분 영상의, 주 방향 성분을 계산하여 보정\
=> 보정된 사각형 영역을 4*4 구역으로 분할=> 각 구역에서의 8방향 히스토그램 구함\
=> 4*4*8 = 128 차원의 실수 벡터 \

실수 기술자는 보통 L2 norm을 사용하여 유사도 판단\



이진 기술자(Binary descriptor)\
Binary Test를 이용하여 부분 영상의 특징 기술\
보통 numpy.uint8 자료형을 사용하여 비트 단위로 영상 특징 정보를 저장하는 방식\

이진 기술자를 사용하는 알고리즘 : AKAZE, ORB, BRIEF 등\
이진 기술자는 해밍거리(Hamming distance)를 사용하여 유사도 판단\
=> 해밍 거리는 곱집합 위에 정의되는 거리함수\
대략 같은 거리 두 문자열에서, 같은 위치에서 서로 다른기호들이 몇개인지 센다

특징점 기술자 계산 함수\
cv2.Feature2D.compute(image,keypoints,descriptors=None)= > keypoints, descriptors\

image: 입력영상\
keypoints: 검출된 특징점 정보, cv2.KeyPoint 객체의 리스트\
descriptors: 특징점 기술자 행렬\

특징점 검출 및 기술자 계산 함수\
cv2.Feature2D.detectAndCompute(image,mask=None,descriptors=None)=>keypoints,descriptors\

image : 입력 영상\
mask: 마스크 영상\
keypoints: 검출된 특징점 정보, cv2.KeyPoint객체의 리스트\
descriptors: 특징점 기술자 행렬

In [8]:
import sys 
import numpy as np
import cv2 

# 입력 영상 불러오기
src1 = cv2.imread('graf1.png',cv2.IMREAD_GRAYSCALE)
src2 = cv2.imread('graf3.png',cv2.IMREAD_GRAYSCALE)

if src1 is None or src2 is None:
    print('Image load failed!')
    sys.exit()

# 어떤 방법으로 특징점을 얻을것인지???
# 특징점 알고리즘 객체 생성(KAZE, AKZE, ORB 등)

# cv2.Feature2D.detect(image, mask=None) => keypoints(특징점 좌표)\

feature = cv2.KAZE_create()
#feature = cv2.AKAZE_create()
#feature = cv2.ORB_create()

# 특징점 검출 및 기술자 계산

#특징점 좌표 얻기 
kp1 = feature.detect(src1)

#기술자 계산
# cv2.Feature2D.compute(image,keypoints,descriptors=None)= > keypoints, descriptors\

_, desc1 = feature.compute(src1,kp1)

# 여기서는 특징점 좌표와 기술자 한꺼번에 계산하는 함수이용
kp2,desc2 = feature.detectAndCompute(src2,None)

#기술자 배열의 사이즈와 타입 확인

print('desc1.shape:', desc1.shape)
print('desc1.dtype:', desc1.dtype)
print('desc2.shape:', desc2.shape)
print('desc2.dtype:', desc2.dtype)

# 검출된 특징점 출력 영상 생성
dst1 = cv2.drawKeypoints(src1,kp1,None,
                        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
dst2 = cv2.drawKeypoints(src2,kp2,None,
                        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


# 이미지로 확인
cv2.imshow('dst1',dst1)
cv2.imshow('dst2',dst2)
cv2.waitKey()
cv2.destroyAllWindows()



desc1.shape: (3159, 64)
desc1.dtype: float32
desc2.shape: (3625, 64)
desc2.dtype: float32


5. 특징점 매칭\

특징점 매칭(feature point matching)\
=> 두 영상에서 추출한 특징점 기술자를 비교하여 서로 유사한 기술자를 찾는 작업

특징 벡터 유사도 측정 방법\
실수 특징 벡터: L2 norm 사용\
이진 특징 벡터: 해밍거리 사용\

특징점 매칭 알고리즘 객체 생성\
cv2.BFMatcher_create(, normType=None, crossCheck=None)=> retval\

normType: 특징점 기술자 거리 계산 방식 지정, 기본값 cv2.NORM_L2

cv2.NORM_L2: L2 노름 사용\
cv2.NORM_HAMMING:해밍 거리 사용\
cv2.NORM_HAMMING2: 두 비트를 한단위로 취급하여 해밍거리 계산\
crossCheck:이 값이 True 이면 양방향 매칭 결과가 같은경우만 반환함\


특징점 검출 알고리즘 객체 생성\
cv2.DescriptorMatcher.match(queryDescriptors, trainDescriptors, mask=None)=> matches\

queryDescriptors:(기준영상 특징점) 질의 기술자 행렬\
trainDescriptors:(대상영상 특징점)학습기술자 행렬\
mask: 매칭 진행 여부를 지정하는 행렬마스크\
matches:매칭 결과\
=>cv2.DMatch 객체의 리스트

특징점 매칭 결과 영상 생성\
cv2.drawMatches(img1, keypoints1,img2,keypoints2,matches1to2, matchColor=None, singlePointColor=None,matchesMast=None,flags=None)\
=>

In [ ]:
cv2.d